In [ ]:
import os
import pdfplumber
import warnings
from pymongo import MongoClient
from sklearn.metrics.pairwise import cosine_similarity
from langchain.embeddings import OpenAIEmbeddings
from langchain.agents import Tool
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.memory import ConversationBufferMemory
from langchain.agents import ConversationalAgent, AgentExecutor

# Suppress deprecation warnings
warnings.filterwarnings("ignore", message=".*The method `Chain.run` was deprecated.*")
warnings.filterwarnings("ignore", message=".*The method `Chain.__call__` was deprecated.*")

# Set API keys
os.environ["OPENAI_API_KEY"] = "your_key"
os.environ["SERPAPI_API_KEY"] = "your_key"

# Tool 1: SERPAPI Tool
search_tool = Tool(
    name="Search Top Skills",
    func=SerpAPIWrapper().run,
    description="Use this about the recent Survey.Use this to search the internet for recent or real-time information, such as LinkedIn profiles, company websites, news articles, or any online content the LLM can't answer directly",
    return_direct=False
)

# Tool 2: Document RAG Tool

def embedding_based_job_description_rag(skills_query: str, folder='JobDescriptions', top_k=3):
    embedding_model = OpenAIEmbeddings()
    query_embedding = embedding_model.embed_query(skills_query)
    matches = []

    for filename in os.listdir(folder):
        if filename.endswith(".pdf"):
            path = os.path.join(folder, filename)
            with pdfplumber.open(path) as pdf:
                content = "\n".join([page.extract_text() or "" for page in pdf.pages])
                if not content.strip():
                    continue
                content_embedding = embedding_model.embed_query(content)
                similarity = cosine_similarity([query_embedding], [content_embedding])[0][0]
                company = filename.replace(".pdf", "")
                location = "Not found"
                for line in content.split("\n"):
                    if "location" in line.lower():
                        location = line.strip()
                        break
                matches.append({"company": company, "location": location, "score": round(similarity, 4)})

    matches.sort(key=lambda x: x["score"], reverse=True)
    top_matches = matches[:top_k]
    if not top_matches:
        return "No relevant companies found."
    return "Top Matched Companies:\n" + "\n".join([f"{m['company']} - {m['location']} (Score: {m['score']})" for m in top_matches])

document_rag_tool = Tool(
    name="Job Description Matcher",
    func=embedding_based_job_description_rag,
    description="Scans job description PDFs for companies needing given skills, and extracts location.",
    return_direct=True
)

# Tool 3a: Candidate Matcher

def fetch_best_matching_candidate(query: str, mongo_uri="mongodb://localhost:27017/", db_name="documentDB", collection_name="extracted_info", top_k=3):
    client = MongoClient(mongo_uri)
    db = client[db_name]
    collection = db[collection_name]
    docs = list(collection.find({"city_skill_embedding": {"$exists": True}}))
    if not docs:
        return "No embedded candidate data found."

    embedding_model = OpenAIEmbeddings()
    query_embedding = embedding_model.embed_query(query)

    similarities = []
    for doc in docs:
        score = cosine_similarity([query_embedding], [doc["city_skill_embedding"]])[0][0]
        similarities.append((score, doc))
    similarities.sort(reverse=True, key=lambda x: x[0])

    top_matches = []
    for score, doc in similarities[:top_k]:
        top_matches.append({
            "name": doc.get("name"),
            "city": doc.get("city"),
            "skills": doc.get("skills"),
            "email": doc.get("email"),
            "phone": doc.get("phone"),
            "score": round(score, 4),
            "filename": doc.get("filename")
        })
    return top_matches

match_candidate_tool = Tool(
    name="Candidate Matcher",
    func=lambda query: str(fetch_best_matching_candidate(query, top_k=3)),
    description="Finds top 3 candidates matching skills and city using MongoDB embeddings.",
    return_direct=True
)

# Tool 3b: Resume Fetcher

def fetch_candidate_resume(filename: str, mongo_uri="mongodb://localhost:27017/", db_name="documentDB", collection_name="extracted_info"):
    client = MongoClient(mongo_uri)
    db = client[db_name]
    collection = db[collection_name]
    doc = collection.find_one({"filename": filename})
    if doc and "raw_text" in doc:
        return doc["raw_text"]
    elif doc and "content" in doc:
        return doc["content"]
    else:
        return "Resume content not found."

resume_tool = Tool(
    name="Candidate Resume Fetcher",
    func=fetch_candidate_resume,
    description="Fetches the full resume of a candidate given their PDF filename.",
    return_direct=True
)

# Memory-aware agent setup
llm = ChatOpenAI(temperature=0, model="gpt-4")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent_chain = ConversationalAgent.from_llm_and_tools(
    llm=llm,
    tools=[search_tool, document_rag_tool, match_candidate_tool, resume_tool],
    verbose=False
)

agent = AgentExecutor.from_agent_and_tools(
    agent=agent_chain,
    tools=[search_tool, document_rag_tool, match_candidate_tool, resume_tool],
    memory=memory,
    verbose=False
)

# Chatbot Interface

def chatbot_agentic_rag():
    print("Agentic RAG Chatbot with Memory is running! Type 'exit' to terminate.")
    while True:
        user_query = input("You: ")
        if user_query.strip().lower() == "exit":
            print("Chatbot session terminated.")
            break
        try:
            response = agent.run(user_query)
            print(f"Bot: {response}")
        except Exception as e:
            print(f"Error: {e}")

if __name__ == "__main__":
    chatbot_agentic_rag()


Agentic RAG Chatbot with Memory is running! Type 'exit' to terminate.


You:  what are the top AI library or tool skills as per recent survey


Bot: Based on the recent survey, the top AI libraries and tools include TensorFlow, PyTorch, Keras, Scikit-learn, and Pandas. These libraries and tools are widely used in the field of AI for various tasks such as data processing, model training, and deployment.


You:  Which companies from my job folder need these skills?


Bot: Top Matched Companies:
DeepAI_Solutions - Location: Bengaluru, India (Score: 0.7933)
AutoAI_Systems - Location: Hyderabad, India (Score: 0.7899)
VisionNext - Location: Hyderabad, India (Score: 0.7762)


...

Bot: I found several social media profiles that could potentially belong to Dr. T.K. Senthil Kumar. Here are some of them:

1. Instagram: Senthil Kumar (@dopkksenthilkumar) - Director of Photography, Oscar awards (Jury member)
2. Instagram: Senthil Kumar T K (@tksenthilkumar)
3. Facebook: DR.TK, Chennai, India. 2424 likes. They pave a way for Learning to all kind of people who are all interested to develop the Nation.

Please note that these profiles may not necessarily belong to the same Dr. T.K. Senthil Kumar you are looking for, as there could be multiple individuals with the same name.


You:  I want his linkedin profile


Bot: I found a LinkedIn profile that seems to match Dr. T.K. Senthil Kumar. The profile describes him as a Senior Data Scientist, Computer Vision Expert, AI & ML Educator, and Subject Matter Expert at L&T EduTech. He is also mentioned as a former Data Scientist at NYBL. Please note that LinkedIn profiles are not publicly accessible without an account, so you would need to search for his name on LinkedIn directly to view the full profile.


You:  Can u get his personal information from his online data ?


Bot: I'm sorry, but it's not appropriate or ethical to seek out or provide personal information about individuals without their consent. It's important to respect privacy and adhere to legal guidelines when using online data.
